In [1]:
year = 2000

In [1]:
from requests import get 
from bs4 import BeautifulSoup
import os
import time
import pandas as pd
import numpy as np
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/")
os.mkdir("IMDB"+str(year))
os.chdir("IMDB"+str(year))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
file_path = 'movie_list_'+str(year)+'.csv'

In [2]:
imdb_url = 'https://www.imdb.com'
home_url = 'https://www.imdb.com'

base_url = "https://www.imdb.com/search/title/?release_date=%d-01-01,%d-12-31&sort=num_votes,desc&start=1"%(year, year)


In [3]:
base_url

'https://www.imdb.com/search/title/?release_date=2000-01-01,2000-12-31&sort=num_votes,desc&start=1'

In [0]:
def movie_catalog_pages(base_url):
    response = None
    try:
        response = get(base_url)
    except:
        print("Not loaded "+ base_url)
    
    return response

def catalog_info(html_soup):
    """
    This function return the release date interval and number of movies
    there are three 
    """
    dates = re.findall(r'\w+ (\d+-\d+-\d+) and (\d+-\d+-\d+)', html_soup.find('div', class_='redesign').h1.text)
    number_of_movie =  html_soup.find('div', class_='desc').span.text
    number_of_movie = int(re.findall(r'\d+\-\d+ of (\w+\,\w+) titles',number_of_movie)[0].replace(',',''))
    
    return dates[0][0], dates[0][1], number_of_movie
    
    
def page_navigator(home_url, html_soup):
    """
    This function returns:
    the address of the next page (if any) if the page do not exist it returns error
    previous page (if any) number  if the page do not exist it returns False
    """

    movie_navigator =  html_soup.find('div', class_='desc')
    if (movie_navigator.find('a',class_='lister-page-next next-page') != None):
        movie_next = movie_navigator.find('a',class_='lister-page-next next-page')
        next_page = home_url + movie_next['href']
    else: 
        exit(1)
        
    if (movie_navigator.find('a',class_='lister-page-prev prev-page') != None):
        movie_prev = movie_navigator.find('a',class_='lister-page-prev prev-page')
        prev_page = home_url + movie_prev['href']
    else: 
        prev_page =  False
    
    return next_page, prev_page
    
def appender(data_list, info):
    """
    Append the info
    if the info is None appends NaN
    """
    if (info == None):
        data_list.append(np.NaN)
    else:
        data_list.append(info)
        

def file_to_csv(data, file_path, status = 'a'):
    if status != 'a':
        cols = ['No.','Movie','Year','IMDB_rating','Number of votes','Meta favor',
                'Meta score','Certificate','Runtime','Genre','Gross Exact Value',
                'Gross Approximation','movie_url']
        headers = pd.DataFrame([], columns=cols)
        headers.to_csv(file_path, index=False)
        
    else:
        data.to_csv(file_path, header=False, mode = 'a')
        


    
            

       




In [0]:
def logger_file(file_path, data_txt):
    with open(file_path, 'a') as f:
        f.write(data_txt+'\n')

In [0]:
def catalog_data_picker(imdb_url, html_soup):
    """
    
    """
    numbers = []
    name = []
    year = []
    IMDB_rating = []
    certificate = []
    runtime = []
    genre = []
    num_votes = []
    gross_with_unit = []
    gross_exact = []
    movie_address = []
    meta_score = []
    meta_favor = []
    
    movies = html_soup.find_all('div', class_='lister-item mode-advanced')
    for movie in movies:
        
        try:
            #=========================      year of release
            temp = movie.h3.find('span', class_ = 'lister-item-year')
            if (temp == None):
                year.append(np.NaN)
            else:
                data_str = re.findall(r'\w*\s*\((\d+\d+\d+\d+)\)',temp.text)       
                if data_str != []:
                    year.append(data_str[0])
                else:
                    continue            




            #=========================      No, of the movie
            temp = movie.h3.span
            if (temp == None):
                numbers.append(np.NaN)
            else:
                numbers.append(int(temp.text.replace('.','').replace(',','')))


            #=========================      name of the movie
            temp = movie.h3.a
            if (temp == None):
                name.append(np.NaN)
            else:
                name.append(temp.text)



            #========================       movie url
            temp = movie.h3.a['href']
            if (temp == None):
                movie_address.append(np.NaN)
            else:
                movie_address.append(temp)


            #======================      IMDB rating
            movie_rating = movie.find('div', class_ = 'ratings-bar').strong
            temp = movie_rating
            if (temp == None):
                IMDB_rating.append(np.NaN)
            else:
                IMDB_rating.append(temp.text)
                #======================      meta score

                if movie_rating.find('div', class_ = 'inline-block ratings-metascore') is None :
                    meta_score.append(np.NaN)
                    meta_favor.append(np.NaN)
                else:
                    movie_meta_score = movie_rating.find('div', class_ = 'inline-block ratings-metascore')
                    meta_score.append(movie_meta_score.span['class'][-1])
                    meta_favor.append(movie_meta_score.span.text.strip())






            #=======================       number of vote & gross

            g_and_v = movie.find('p', class_='sort-num_votes-visible').find_all('span', attrs = {'name':'nv'})
            appender(num_votes, int(movie.find('p', class_='sort-num_votes-visible').find_all('span', attrs = {'name':'nv'})[0]['data-value']))


            if (len(g_and_v) > 1):
                appender(gross_with_unit, movie.find('p', class_='sort-num_votes-visible').find_all('span', attrs = {'name':'nv'})[1].text)
                appender(gross_exact, movie.find('p', class_='sort-num_votes-visible').find_all('span', attrs = {'name':'nv'})[1]['data-value']) 
            else:
                appender(gross_with_unit, np.NaN)
                appender(gross_exact,np.NaN)

            #====================         certificate
            temp = movie.p.find('span', class_="certificate")
            if (temp == None):
                certificate.append(np.NaN)
            else:
                certificate.append(temp.text)


            #====================         runtime
            temp = movie.p.find('span', class_="runtime")
            if (temp == None):
                runtime.append(np.NaN)
            else:
                runtime.append(temp.text)


            #====================         genre 
            temp = movie.p.find('span', class_="genre")
            if (temp == None):
                genre.append(np.NaN)
            else:
                genre.append(temp.text.replace("\n",'').strip())
        except:
            with open('log','w') as f:
                print("raise a problem "+str(movie.h3.a))
                f.write(movie.h3.a+'\n\n\n\n')

             

        #=================== Creating data table
        
        catalog_dataframe = pd.DataFrame({'No.':numbers,
                                     'Movie':name,
                                     'Year':year,
                                     'IMDB_rating':IMDB_rating, 
                                     'Number of votes':num_votes,
                                     'Meta favor':meta_favor,
                                     'Meta score':meta_score,
                                     'Certificate':certificate,
                                     'Runtime':runtime,
                                     'Genre':genre,
                                     'Gross Exact Value':gross_exact,
                                     'Gross Approximation':gross_with_unit,
                                     'movie_url':movie_address})
        
    return catalog_dataframe
    

In [0]:
dataframe_header  = pd.DataFrame(columns =('No.','Movie','Year','IMDB_rating','Number of votes',
                           'Meta favor','Meta score','Certificate','Runtime',
                           'Genre','Gross Exact Value','Gross Approximation','movie_url'))

with open(file_path, 'w') as f:
    dataframe_header.to_csv(f)
    
dataframe_of_movies = dataframe_header

In [0]:
def catalog_pager(file_path, home_url, base_url, max_page = "TheEnd"):
       
    while True:
        #--------------- get the response
        try:
            response = movie_catalog_pages(base_url)
        except:
            logger_file("failed_page_catalog.log", base_url)
            print("failed crawed file", base_url)
            continue

        #------------- create soup and read data



        response = movie_catalog_pages(base_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        catalog_page = catalog_data_picker(home_url, html_soup)

        with open(file_path, 'a') as f:
            catalog_page.to_csv(f, header=False)
        
        #------------ next page
        try:
            next_page, end_page = page_navigator(home_url, html_soup)
        except:
            print("end of pages")
            break
        print("crawling This: ",base_url)
        
        base_url = next_page
        
        # lag the crawler
        time.sleep(np.random.random()+0.2)

In [0]:
catalog_pager(file_path, home_url, base_url, max_page = "TheEnd")

crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=1
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=51
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=101
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=151
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=201
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=251
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=301
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,2018-12-31&sort=num_votes,desc&start=351
crawling This:  https://www.imdb.com/search/title/?release_date=2018-01-01,

TypeError: ignored